# Single Determinant

Here the array MO contains a set of orthogonal (generated by doing qr decomposition of a random matrix) column vectors that can be throught of as molecular orbitals.

Determinants are defined using molecular orbitals. A single determinant $|D_0\rangle$ is defined as containing the first nelec molecular orbitals. 

Walkers are defined in the local basis and let us assume $|w_0\rangle$ contains the first nelec local orbitals as full.

Our job is to calculate the value of $\langle w_0|D_0\rangle$ which is the determiant of an nelec x nelec matrix $A_0$.

In [2]:
import numpy

numpy.random.seed(0)
norbs, nelec = 20, 5

#generate a random matrix of mo coeffs
MO = numpy.random.random((norbs,norbs))
MO = numpy.linalg.qr(MO)[0] #make the columns orthogonal

D0 = 1.*MO[: , 0:nelec]
w0 = numpy.asarray(range(nelec))

detMatrix = D0[w0,:]

detinv = numpy.linalg.inv(detMatrix)
det0   = numpy.linalg.det(detMatrix)
print det0

-0.0038666423824060578


Now let us assume that we will change the walker by exciting an electron in orbital nelec-1 to nelec+1 giving us the walker $|w_1\rangle$. The aim is to calculate the new determinant $\langle w_1|D_0\rangle$.
This can be done by using the determinant lemma

$\langle w_1|D_0\rangle$ = $\langle w_0|D_0\rangle \Tr(bA_0^{-1})$,
where $A_0^{-1}$ is the inverse of the matrix $A_0$, which is the nelec x nelec matrix corresponding to $\langle w_0|D_0\rangle$ and $b$ is the row matrix introduced by the walker $|w_1\rangle$.

In [3]:
w1 = 1*w0
w1[-1] = nelec+1

#required answer
print numpy.linalg.det(D0[w1,:])

#Sherman-Morrison formula for calculating it
print det0*numpy.dot(D0[nelec+1, :], detinv[:,nelec-1])

-0.0013837690769244247
-0.0013837690769244238


Now lets assume that the we have a multideterminantal wavefunction, in this example lets just assume we have 3 determinants $|D_0\rangle + x_1 |D_1\rangle + x_2 |D_2\rangle$.

Where Determinant $|D_1\rangle$ is constructed by exciting an electron from molecular orbital nelec-1 to nelec+1 in $|D_0\rangle$ and determinant $|D_2\rangle$ is constructed by exciting an electron from molecular orbital nelec-2 to nelec+3 in $|D_1\rangle$

\begin{align}
\langle w_0|\Phi\rangle =& \langle w_0|D_0\rangle + x_1\langle w_0|D_1\rangle + x_2\langle w_0|D_2\rangle \\
\partial_{\nu} \ln(\langle w_0|\Phi\rangle) =& \partial_{\nu} \ln\left(\langle w_0|D_0\rangle) + x_1 \langle w_0|D_1\rangle) + x_2 \langle w_0|D_2\rangle\right) \\
\partial_{\nu} \ln(\langle w_0|\Phi\rangle) =& \partial_{\nu} \ln\left(det(A_0) + x_1 det(A_1) + x_2 det(A_2)\right)\\
=& \frac{\partial_{\nu} \left(det(A_0) + x_1 det(A_1) + x_2 det(A_2)\right)}{det(A_0) + x_1 det(A_1) + x_2 det(A_2)}\\
=&\frac{\left(adj(A_0)_{\nu} + x_1 adj(A_1)_{\nu} + x_2 adj(A_2)_{\nu}\right)}{det(A_0) + x_1 det(A_1) + x_2 det(A_2)}\\
=&\frac{\left(det(A_0) (A_0^{-1})_{\nu.T} + x_1 det(A_1) (A_1^{-1})_{\nu.T} + x_2 det(A_2) (A_2^{-1})_{\nu.T}\right)}{det(A_0) + x_1 det(A_1) + x_2 det(A_2)}\\
\end{align}
where $\nu$ is just a position e.g. $ij$, in which case $\nu.T$ is $ji$.

In [4]:
from numpy.linalg import det as DET

D1 = 1.*MO[: , [0,1,2,3,6]]
D2 = 1.*MO[: , [0,1,2,6,8]]

x0, x1, x2 = 1., 1., 1.
A00, A01, A02 = D0[w0,:], D1[w0, :], D2[w0, :]
print numpy.linalg.det(x0*A00 + x1*A01 + x2*A02)
#simply use formula shown above

A10, A11, A12 = D0[w1,:], D1[w1, :], D2[w1, :]
print DET(x0*A10) + DET(x1*A11) + DET(x2*A12), (DET(x0*A10) + DET(x1*A11) + DET(x2*A12))/(DET(x0*A00) + DET(x1*A01) + DET(x2*A02))

A00inv, A01inv, A02inv = numpy.linalg.inv(A00), numpy.linalg.inv(A01), numpy.linalg.inv(A02)
detA00, detA01, detA02 = numpy.linalg.det(A00), numpy.linalg.det(A01), numpy.linalg.det(A02) 


Gamma = numpy.zeros((norbs, nelec))
Gamma[0:nelec    , :] += x0*detA00*A00inv 
Gamma[[0,1,2,3,6], :] += x1*detA01*A01inv   
Gamma[[0,1,2,6,8], :] += x2*detA02*A02inv 
#Gamma = x0*A00inv + x1*(detA01/detA00)*A01inv   + x2*(detA02/detA00)*A02inv 
print  numpy.trace( numpy.dot( (MO[w0, :] - MO[w1, :]), Gamma))
print  1+numpy.dot( (MO[nelec+1, :]-MO[nelec-1, :]), Gamma[:,nelec-1])/(DET(x0*A00) + DET(x1*A01) + DET(x2*A02))
print  numpy.dot( (MO[nelec+1, :]), Gamma[:,nelec-1])/(DET(x0*A00) + DET(x1*A01) + DET(x2*A02))

-0.1404576015283056
-0.017496567367822655 -5.832804266791937
0.020496250979260864
-5.832804266791936
-5.8328042667919355


In [3]:
#lets say we want to replace a col of a
col = numpy.random.random((5,))
colIndex = 3
anew = 1.*a
anew[:, colIndex] = 1.*col

print numpy.linalg.det(anew)

#Use sherman Morrison formula
print numpy.linalg.det(a)*(numpy.dot( ainv[colIndex,:], col )), (numpy.dot( ainv[colIndex,:], col ))

0.004270646756593412
0.00427064675659342 0.44217716328855206


In [46]:
numpy.random.seed(5)
a = numpy.random.random((5,5))
ainv = numpy.linalg.inv(a)
row = numpy.random.random((5,))
col = numpy.random.random((1,5))

colIndex = 3
rowIndex = 3
def trace(A, B):
    trace = 0.
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            trace += A[i,j]*B[i,j]
    return trace

anew = 1.*a
#anew[rowIndex,:] = 1.*row
anew[:, colIndex] = 1.*numpy.random.random((1,5))
anew[:, colIndex+1] = 1.*numpy.random.random((1,5))

B = anew-a

#Filippi formula
derivGamma = 1.*ainv.T
print numpy.linalg.det(anew)/numpy.linalg.det(a), 1+trace(derivGamma, B)

-2.5136295349048154 -2.115823465040155
